In [2]:
import os
import sqlite3
from pathlib import Path
import pandas as pd
# Opción A: ruta absoluta hard-codeada
DB_PATH = "/home/navi/Desktop/archivos/DS/eco/Proyecto_macro/db/proyectomacro.db"

# Opción B (más flexible): lee la ruta de una variable de entorno
# 1. En tu terminal: export PROJECT_DB="/home/…/proyectomacro.db"
# 2. En el script:
# DB_PATH = os.environ.get("PROJECT_DB", "/home/navi/.../proyectomacro.db")

# Con pathlib para asegurar que es un string correcto
DB_FILE = Path(DB_PATH)
conn = sqlite3.connect(str(DB_FILE))
pib_g = pd.read_sql_query("""
    SELECT año, crecimiento
    FROM   tasa_crecimiento_pib
""", conn)
prices = pd.read_sql_query("""
    SELECT *
    FROM precio_oficial_minerales
""", conn)
df = (
    pib_g                       # y-variable
    .merge(prices, on="año")    # X-variables
    .dropna()                   # keep only complete years
)
df.set_index("año", inplace=True)
df.head()


,crecimiento,zinc,estaño,oro,plata,antimonio,plomo,wolfram,cobre,bismuto,cadmio,manganeso
año,,,,,,,,,,,,
1980,0.57,0.36,7.61,610.06,20.96,1750.98,0.41,141.41,0.96,2.41,2.50,0.84
1981,0.28,0.41,6.39,466.83,10.84,1519.69,0.33,148.33,0.79,2.05,2.50,0.84
1982,-3.94,0.39,5.74,374.46,7.79,1186.02,0.25,105.79,0.66,1.48,1.34,0.84
1983,-4.04,0.37,5.88,415.93,11.43,1035.43,0.19,77.95,0.70,1.57,1.13,0.84
1984,-0.20,0.46,5.54,359.29,8.15,1810.04,0.20,78.41,0.63,3.64,1.33,0.84


In [4]:
df.loc[1980:1990]

,crecimiento,zinc,estaño,oro,plata,antimonio,plomo,wolfram,cobre,bismuto,cadmio,manganeso
año,,,,,,,,,,,,
1980,0.57,0.36,7.61,610.06,20.96,1750.98,0.41,141.41,0.96,2.41,2.50,0.84
1981,0.28,0.41,6.39,466.83,10.84,1519.69,0.33,148.33,0.79,2.05,2.50,0.84
1982,-3.94,0.39,5.74,374.46,7.79,1186.02,0.25,105.79,0.66,1.48,1.34,0.84
1983,-4.04,0.37,5.88,415.93,11.43,1035.43,0.19,77.95,0.70,1.57,1.13,0.84
1984,-0.20,0.46,5.54,359.29,8.15,1810.04,0.20,78.41,0.63,3.64,1.33,0.84
1985,-1.68,0.39,5.32,316.15,6.12,1648.62,0.18,64.50,0.63,4.94,1.24,1.24
1986,-2.57,0.36,2.57,363.25,5.44,1810.04,0.18,44.85,0.61,2.70,0.91,1.34
1987,2.46,0.38,3.00,418.28,6.21,1902.56,0.25,44.02,0.73,3.19,1.62,1.34
1988,2.91,0.52,3.23,438.71,6.51,2202.76,0.29,52.33,1.06,5.37,6.80,1.34


In [7]:
import statsmodels.formula.api as smf

model = smf.ols(
    "crecimiento ~ estaño + zinc + oro + plata + antimonio + plomo",
    data=df
).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:            crecimiento   R-squared:                       0.520
Model:                            OLS   Adj. R-squared:                  0.442
Method:                 Least Squares   F-statistic:                     6.688
Date:                Fri, 13 Jun 2025   Prob (F-statistic):           7.46e-05
Time:                        16:35:51   Log-Likelihood:                -95.754
No. Observations:                  44   AIC:                             205.5
Df Residuals:                      37   BIC:                             218.0
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.6840      0.975      2.753      0.0